In [1]:
!git clone https://github.com/facebookresearch/dlrm.git
!touch ./dlrm/__ini__.py

fatal: destination path 'dlrm' already exists and is not an empty directory.


In [2]:
# experiment params
num_train_trials = 5
num_train_warmups = 1
num_jit_trials = 10
num_jit_warmups = 2
num_inference_trials = 10000
num_inference_warmups = 1000

In [3]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd()/Path("dlrm")))
print(sys.path)

from dlrm.dlrm_s_pytorch import *

class Args:
    arch_sparse_feature_size=16
    arch_embedding_size='4-3-2'
    arch_mlp_bot='13-128-64-32'
    arch_mlp_top='80-128-64-1'
    arch_interaction_op='dot'
    arch_interaction_itself=False
    weighted_pooling=None
    loss_function='mse'
    md_flag=False
    md_threshold=200
    md_temperature=0.3
    md_round_dims=False
    qr_flag=False
    qr_threshold=200
    qr_operation='mult'
    qr_collisions=4
    activation_function='relu'
    loss_function='bce'
    loss_weights='1.0-1.0'
    loss_threshold=0.0
    round_targets=True
    data_size=1
    num_batches=0
    data_generation='random'
    rand_data_dist='uniform'
    rand_data_min=0
    rand_data_max=1
    rand_data_mu=-1
    rand_data_sigma=1
    data_trace_file='./input/dist_emb_j.log'
    data_set='kaggle'
    raw_data_file='./data_kaggle/train.txt'
    processed_data_file='./data_kaggle/kaggleAdDisplayChallenge_processed.npz'
    data_randomize='total'
    data_trace_enable_padding=False
    max_ind_range=-1
    data_sub_sample_rate=0.0
    num_indices_per_lookup=10
    num_indices_per_lookup_fixed=False
    num_workers=0
    memory_map=False
    mini_batch_size=8192
    nepochs=1
    learning_rate=0.1
    print_precision=5
    numpy_rand_seed=123
    sync_dense_params=True
    optimizer='sgd'
    dataset_multiprocessing=True
    inference_only=False
    quantize_mlp_with_bit=32
    quantize_emb_with_bit=32
    save_onnx=False
    use_gpu=True
    local_rank=-1
    dist_backend=''
    print_freq=1024
    test_freq=1024
    test_mini_batch_size=8192
    test_num_workers=-1
    print_time=False
    print_wall_time=False
    debug_mode=True
    enable_profiling=False
    plot_compute_graph=False
    tensor_board_filename='run_kaggle_pt'
    save_model=''
    load_model=''
    mlperf_logging=False
    mlperf_acc_threshold=0.0
    mlperf_auc_threshold=0.0
    mlperf_bin_loader=False
    mlperf_bin_shuffle=False
    mlperf_grad_accum_iter=1
    lr_num_warmup_steps=0
    lr_decay_start_step=0
    lr_num_decay_steps=0
    nbatches=1000

global args
args = Args()

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/achilibe/Code/2JITsu/venv/lib/python3.10/site-packages', '/home/achilibe/Code/2JITsu/ViT/torch/vit_pytorch', '/home/achilibe/Code/2JITsu/BERT/torch/benchmark/torchbenchmark/models/BERT_pytorch', '/home/achilibe/Code/2JITsu/DLRM/torch/dlrm']
Unable to import mlperf_logging,  No module named 'mlperf_logging'


2024-10-22 14:15:24.357723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 14:15:24.365017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 14:15:24.372967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 14:15:24.375364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 14:15:24.381914: I tensorflow/core/platform/cpu_feature_guar

In [4]:
import tqdm
import time
import torch
import torch._dynamo.utils
from torch._dynamo.eval_frame import _debug_get_cache_entry_list

dlrm = DLRM_Net(
    16,
    np.fromstring(args.arch_embedding_size, dtype=int, sep="-"),
    np.fromstring(args.arch_mlp_bot, dtype=int, sep="-"),
    np.fromstring(args.arch_mlp_top, dtype=int, sep="-"),
    arch_interaction_op='cat',
    arch_interaction_itself=False,
    sigmoid_bot=-1,
    sigmoid_top=2
)

import os

os.environ['TORCHINDUCTOR_FORCE_DISABLE_CACHES'] = '1'

def time_jit(num_jit_runs: int, num_jit_warmups: int, model: nn.Module) -> float:
    times = []
    for i in tqdm.trange(1, num_jit_runs + 1):
        start_jit_time = time.time()
        
        jit_model = torch.compile(
            model, 
            options={"triton.cudagraphs": True}, 
            fullgraph=True,
            backend='cudagraphs'
        )

        end_jit_time = time.time()
        #print(_debug_get_cache_entry_list(jit_model._torchdynamo_orig_callable))
        torch.compiler.reset()
        torch._dynamo.reset()
        torch.cuda.empty_cache()
        #print(_debug_get_cache_entry_list(jit_model._torchdynamo_orig_callable))
        if i >= num_jit_warmups:
            times.append(end_jit_time - start_jit_time)

    return sum(times) / len(times)
    
print(torch._dynamo.list_backends())
average_jit_time = time_jit(num_jit_trials, num_jit_warmups, dlrm)
print(f"Average JIT time: {average_jit_time}")

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']


100%|████████████████████████████████████████| 10/10 [00:00<00:00, 21.28it/s]

Average JIT time: 0.00014212396409776475


In [5]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mrkmakr/criteo-dataset")

In [6]:
import dlrm.dlrm_data_pytorch as dp
import numpy as np

ln_emb = np.fromstring(args.arch_embedding_size, dtype=int, sep="-")
m_den = np.fromstring(args.arch_mlp_bot, dtype=int, sep="-")[0]

use_gpu = False
global device
if use_gpu: 
    device = torch.device("cuda") 
else: 
    device = torch.device("cpu") 

train_data, train_ld, test_data, test_ld = dp.make_random_data_and_loader(
    args, ln_emb, m_den
)

parameters = (
    dlrm.parameters()
    if ext_dist.my_size == 1
    else [
        {
            "params": [p for emb in dlrm.emb_l for p in emb.parameters()],
            "lr": args.learning_rate,
        },
        # TODO check this lr setup
        # bottom mlp has no data parallelism
        # need to check how do we deal with top mlp
        {
            "params": dlrm.bot_l.parameters(),
            "lr": args.learning_rate,
        },
        {
            "params": dlrm.top_l.parameters(),
            "lr": args.learning_rate,
        },
    ]
)
optimizer = torch.optim.SGD(parameters, lr=args.learning_rate)
lr_scheduler = LRPolicyScheduler(
    optimizer,
    args.lr_num_warmup_steps,
    args.lr_decay_start_step,
    args.lr_num_decay_steps,
)

In [7]:
# The following function is a wrapper to avoid checking this multiple times in th
# loop below.
def unpack_batch(b):
    # Experiment with unweighted samples
    return b[0], b[1], b[2], b[3], torch.ones(b[3].size()), None

def dlrm_wrap(X, lS_o, lS_i, use_gpu, device, ndevices=1, dlrm=dlrm):
    with record_function("DLRM forward"):
        if use_gpu:  # .cuda()
            # lS_i can be either a list of tensors or a stacked tensor.
            # Handle each case below:
            if ndevices == 1:
                lS_i = (
                    [S_i.to(device) for S_i in lS_i]
                    if isinstance(lS_i, list)
                    else lS_i.to(device)
                )
                lS_o = (
                    [S_o.to(device) for S_o in lS_o]
                    if isinstance(lS_o, list)
                    else lS_o.to(device)
                )
                print("tensors")
        return dlrm(X.to(device), lS_o, lS_i)

def loss_fn_wrap(Z, T, use_gpu, device):
    with record_function("DLRM loss compute"):
        if args.loss_function == "mse" or args.loss_function == "bce":
            return dlrm.loss_fn(Z, T.to(device))
        elif args.loss_function == "wbce":
            loss_ws_ = dlrm.loss_ws[T.data.view(-1).long()].view_as(T).to(device)
            loss_fn_ = dlrm.loss_fn(Z, T.to(device))
            loss_sc_ = loss_ws_ * loss_fn_
            return loss_sc_.mean()

In [8]:
for i, j in enumerate(train_ld):
    print(i)
    print(j)

0
(tensor([[0.6965, 0.2861, 0.2269, 0.5513, 0.7195, 0.4231, 0.9808, 0.6848, 0.4809,
         0.3921, 0.3432, 0.7290, 0.4386]]), tensor([[0],
        [0],
        [0]]), [tensor([1]), tensor([0]), tensor([1])], tensor([[1.]]))


In [15]:
def time_train(num_train_runs: int, num_train_warmups: int, args: Args):
    train_data, train_ld, test_data, test_ld = dp.make_random_data_and_loader(
        args, ln_emb, m_den
    )

    parameters = (
        dlrm.parameters()
        if ext_dist.my_size == 1
        else [
            {
                "params": [p for emb in dlrm.emb_l for p in emb.parameters()],
                "lr": args.learning_rate,
            },
            # TODO check this lr setup
            # bottom mlp has no data parallelism
            # need to check how do we deal with top mlp
            {
                "params": dlrm.bot_l.parameters(),
                "lr": args.learning_rate,
            },
            {
                "params": dlrm.top_l.parameters(),
                "lr": args.learning_rate,
            },
        ]
    )
    optimizer = torch.optim.SGD(parameters, lr=args.learning_rate)
    lr_scheduler = LRPolicyScheduler(
        optimizer,
        args.lr_num_warmup_steps,
        args.lr_decay_start_step,
        args.lr_num_decay_steps,
    )

    X, lS_o, lS_i, T, W, CBPP = unpack_batch(enumerate(train_ld).__next__()[1])
    
    for i in tqdm.trange(num_train_trials):
    
        
        times = []
        start_train_time = time.time()
        
        for i in range(1000):

            # forward pass
            Z = dlrm_wrap(
                X,
                lS_o,
                lS_i,
                use_gpu,  # use gp u
                device,
                ndevices=1,
            )

            # loss
            E = loss_fn_wrap(Z, T, use_gpu, device)

            # compute loss and accuracy
            L = E.detach().cpu().numpy()  # numpy array
            E.backward()

            # optimizer
            optimizer.step()
            lr_scheduler.step()

        end_train_time = time.time()
        if i >= num_train_warmups:
            times.append(end_train_time - start_train_time)

    return sum(times) / len(times)

average_training_time = time_train(num_train_trials, num_train_warmups, 
                                args)
print(f"Average training time: {average_training_time}")

100%|██████████████████████████████████████████| 5/5 [00:02<00:00,  2.24it/s]

Average training time: 0.4381582736968994


In [16]:
def time_inference(num_inference_trials, num_inference_warmups):
    train_data, train_ld, test_data, test_ld = dp.make_random_data_and_loader(
        args, ln_emb, m_den
    )
    
    X, lS_o, lS_i, T, W, CBPP = unpack_batch(enumerate(train_ld).__next__()[1])
    
    times = []
    for i in tqdm.trange(num_inference_trials):
        start_inference_time = time.time()

        dlrm_wrap(
            X,
            lS_o,
            lS_i,
            use_gpu,  # use gp u
            device,
            ndevices=1
        )

        end_inference_time = time.time()
        if i >= num_inference_warmups:
            times.append(end_inference_time - start_inference_time)

    return sum(times) / len(times)

average_inference_time = time_inference(num_inference_trials, num_inference_warmups)
print(f"Average inference time: {average_inference_time}")

100%|███████████████████████████████| 10000/10000 [00:00<00:00, 10985.45it/s]

Average inference time: 8.954938252766927e-05


In [ ]:
# export the python file
!jupyter nbconvert --to script DLRM-Torch.ipynb
!mv DLRM-Torch.py gen_dlrm_torch.py

# write the results to the results.csv file
import csv
with open('results.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Model", "Framework", "Evaluation", "Trials", "Warmups", "Time", "Notes"])
    csvwriter.writerow(["DLRM", "Torch", "train", 
                       num_train_trials, 
                       num_train_warmups, 
                       average_training_time, 
                       ""])
    csvwriter.writerow(["DLRM", "Torch", "inference", 
                       num_inference_trials, 
                       num_inference_warmups, 
                       average_inference_time, 
                       ""])
    csvwriter.writerow(["DLRM", "Torch", "JIT",
                        num_jit_trials,
                        num_jit_warmups,
                        average_jit_time,
                        ""])